## **利用ogr开源库进行矢量文件的读写**


In [1]:
from osgeo import ogr


#### **1. 用python读入矢量文件**

In [2]:
path_kunming = 'data/Section-2/kunming_districts.shp'
path_kunming_out_1 = 'data/Section-4/vector_out_1.shp'
path_kunming_out_2 = 'data/Section-4/vector_out_2.shp'


In [3]:
### 读入矢量文件
ds = ogr.Open(path_kunming, 0)  # 0是只读，1是可写
layer = ds.GetLayer(0)          # 获取矢量文件Layer
num_fea = layer.GetFeatureCount()   ## 统计要素个数
print('Number of feature: ', num_fea)
num_field = layer.GetFeature(0).GetFieldCount()   ## 统计字数个数
print('Number of fields:', num_field)


Number of feature:  14
Number of fields: 8


#### **2.查看矢量文件字段**

In [4]:
### 查看字段名
fields = []
fea = layer.GetFeature(0)
for i in range(0, num_field):
    field = fea.GetFieldDefnRef(i).name
    fields.append(field)
print(fields)


['dt_adcode', 'dt_name', 'ct_adcode', 'ct_name', 'pr_adcode', 'pr_name', 'cn_adcode', 'cn_name']


In [5]:
### 查看字段值
field_name = 'dt_name'
fea = layer.GetFeature(1)
print('Filed value:\n', fea.GetField(field_name))
### 获得所有要素的字段值
for fea in layer:   ##
  print(fea.GetField(field_name))



Filed value:
 盘龙区
五华区
盘龙区
官渡区
西山区
东川区
呈贡区
晋宁区
富民县
宜良县
石林彝族自治县
嵩明县
禄劝彝族苗族自治县
寻甸回族彝族自治县
安宁市


#### **3. 写出矢量文件**

#### 3.1 写出只含有特定字段的矢量文件
1）获取矢量文件图层；2）创建新矢量文件，以此创建数据源、图层、字段。其中新建图层需定义图层名、几何、空间参考。新建字段需先定义字段，定义字段需提供字段名、字段数据类型。3）然后遍历矢量图层要素，提取要素几何，并写入新数据要素中，同时定义新要素字段。
  


In [6]:
field_name = 'dt_name'
ds = ogr.Open(path_kunming, 0)  # 0是只读，1是可写
layer = ds.GetLayer()           # 获取矢量文件图层Layer
driver = ogr.GetDriverByName('ESRI Shapefile')   ## 文件驱动（用于写出文件）
ds_out = driver.CreateDataSource(path_kunming_out_1)  ## 创建数据源DataSource
layer_out = ds_out.CreateLayer('vector_new_1', geom_type=ogr.wkbPolygon, srs=layer.GetSpatialRef())
field_defn = ogr.FieldDefn(field_name, ogr.OFTReal)   ## 定义字段
layer_out.CreateField(field_defn)    ## 新建字段
# fea_defn = layer_out.GetFeature(0).GetDefnRef()    ## layer_out没有要素，故.GetFeature(0)会报错
fea_defn = layer_out.GetLayerDefn()  ## 获得要素定义
### 创建要素（将复制‘昆明市边界_wgs84.shp’文件中要素)
for fea in layer:
    geo = fea.geometry()             ## 获得原矢量要素的几何
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(geo)
    fea_out.SetField(field_name, fea.GetField(field_name))
    layer_out.CreateFeature(fea_out)
ds_out = None       #  ### 保存/关闭 DataSource (!重要)




Warning 1: Value '五华区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '盘龙区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '官渡区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '西山区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '东川区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '呈贡区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '晋宁区' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '富民县' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '宜良县' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '石林彝族自治县' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '嵩明县' of field vector_out_1.dt_name parsed incompletely to real 0.
Warning 1: Value '禄劝彝族苗族自治县' of field vector_out_1.dt_name parsed incomp

#### 3.2 写出只含有呈贡区的矢量文件


In [8]:
ds = ogr.Open(path_kunming, 0)           # 0是只读，1是可写
layer = ds.GetLayer()    # 获取矢量文件Layer
### 
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_out = driver.CreateDataSource(path_kunming_out_2)
layer_out = ds_out.CreateLayer('vector_new_2', geom_type=ogr.wkbPolygon, srs=layer.GetSpatialRef())
fea_defn = layer_out.GetLayerDefn()    ## 获得空要素定义(或空图层定义)

### 创建字段
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (before field creating): ', layer_defn.GetFieldCount())  ### 字段创建前字段数
fea = layer.GetFeature(0)
for i in range(fea.GetFieldCount()):
  field_defn = fea.GetFieldDefnRef(i)
  layer_out.CreateField(field_defn)
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (after field creating): ', layer_defn.GetFieldCount())

### 创建要素（复制‘昆明市边界_wgs84.shp’文件中要素)
for fea in layer:
  if fea.GetField('dt_name') == '呈贡区':
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(fea.geometry())
    for i in range(fea.GetFieldCount()):
      field_value = fea.GetField(i)
      fea_out.SetField(i, field_value)
    layer_out.CreateFeature(fea_out)
ds_out = None        #  ### 保存/关闭 DataSource (重要)



Number of fields (before field creating):  0
Number of fields (after field creating):  8


Warning 1: One or several characters couldn't be converted correctly from UTF-8 to ISO-8859-1.  This warning will not be emitted anymore.


### 作业：写出只含五华区、盘龙区、西山区、官渡区、呈贡区的矢量文件。